In [1]:
import numpy as np
import mne
import pandas as pd
import sys

import csv
from scipy.stats import skew, kurtosis
import pyeeg as p
from numpy import nan
import math


In [2]:
def all_psd(data):
    fs = 256                                
    N = data.shape[1] # total num of points 

    # Get only in postive frequencies
    fft_vals = np.absolute(np.fft.rfft(data))

    n_rows = fft_vals.shape[0]
    n_cols = fft_vals.shape[1]
    psd_vals = np.zeros(shape=(n_rows, n_cols))

    for i in range(n_rows):
        for j in range(n_cols):
            psd_vals[i][j] = (N/fs) * fft_vals[i][j] * fft_vals[i][j];


    # Get frequencies for amplitudes in Hz
    fft_freq = np.fft.rfftfreq(data.shape[1], 1.0/fs)

    # Define EEG bands
    eeg_bands = {'Delta': (0, 4),
                 'Theta': (4, 8),
                 'Alpha': (8, 12),
                 'Beta': (12, 30),
                 'Gamma': (30, 45)}

    # Take the mean of the fft amplitude for each EEG band
    eeg_band_fft = dict()
    psd_vals_list = []
    for band in eeg_bands:  
        freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & 
                           (fft_freq <= eeg_bands[band][1]))[0]
        eeg_band_fft[band] = psd_vals[:,freq_ix]
        for i in range(len(eeg_band_fft[band])): 
            psd_vals_list.append(np.mean(eeg_band_fft[band][i]) * 1000000)
    return psd_vals_list

In [4]:
data = mne.io.read_raw_edf('chb01_03.edf', preload=True)
header = ','.join(data.ch_names)
df = data.to_data_frame()

Extracting EDF parameters from F:\PY\Pro\pro\specto\chb01_03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


<ipython-input-4-cb28b99d03e4>:1: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  data = mne.io.read_raw_edf('chb01_03.edf', preload=True)


In [23]:
seizure_var=1
def add_row(ip_psd, start, end):
    row_to_add = []
    
    ip_psd = ip_psd[:,start:end]
    ip_psd = ip_psd[:][0]
    
    
    p = all_psd(ip_psd)
    p.append(seizure_var)
    
    return p

In [24]:
data = mne.io.read_raw_edf('chb01_03.edf')
header = ','.join(data.ch_names)
df = pd.DataFrame(data[:][0])
df = df.transpose()

Extracting EDF parameters from F:\PY\Pro\pro\specto\chb01_03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-24-e0b81f09e547>:1: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  data = mne.io.read_raw_edf('chb01_03.edf')


In [25]:
start = temp = 2996*256 
duration = 10*256
end = 3036*256

df1 = df.iloc[start:end, :]
df1 = pd.DataFrame(df1)
df1.shape

(10240, 23)

In [26]:

res = pd.DataFrame()

#first iteration run in 'w' mode, all subsequent iteration run in 'a' mode
#with open('test.csv', 'w') as file:
#    writer = csv.writer(file)
while temp < end:    
    row = add_row(data, temp, temp + duration)
    res=res.append(pd.Series(row),ignore_index=True)
#        writer.writerow(row)
    temp += duration

#res.columns = ['Variance', 'Kurtosis', 'Skewness', 'Petrosian Fractal Dimension', 'Hjorth Mobility', 'Hjorth Complexity', 'Spectral Entropy', 'Label']
res

,0,1,2,3,4,5,6,7,8,9,...,106,107,108,109,110,111,112,113,114,115
0,2398.430145,1057.376321,1228.272867,1425.589491,4015.352730,2253.893527,1467.410656,2859.182774,4128.874769,5727.598745,...,32.702493,5.868905,1.389389,1.267681,1.011791,5.608958,28.365478,28.583358,32.702493,1.0
1,5683.760177,3870.849233,1800.316498,955.461104,6227.326422,4427.130113,2142.612872,2960.410795,5583.169827,6398.922857,...,143.669691,11.165056,5.857666,1.767788,2.342996,6.234717,42.395905,79.530479,143.669691,1.0
2,15883.536738,6928.386027,3643.387289,1681.489361,17768.640030,7031.835493,2702.708040,3313.457312,8903.371842,10209.192165,...,207.652680,15.822586,4.673614,1.478758,18.882396,20.171697,30.897911,101.522740,207.652680,1.0
3,12425.395306,4175.535057,2614.413841,2501.161790,15171.964415,5781.670067,3326.714216,3808.665648,9489.501332,9354.724767,...,139.274665,19.535967,2.236787,1.314111,3.419047,21.297453,67.694015,101.309500,139.274665,1.0


In [27]:
from sklearn.ensemble import RandomForestClassifier

In [30]:
rf_cs = RandomForestClassifier()